<a href="https://colab.research.google.com/github/yupcoding1/Autmomatic_email_reply_system/blob/main/email_replier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install langchain-groq
!pip -q install -U langchain_community tiktoken langchainhub
!pip -q install -U langchain langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.2/374.2 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/10

In [ ]:
!pip -q install langchain-chroma bs4
!pip -q install huggingface_hub unstructured sentence_transformers

In [ ]:
import os
from pprint import pprint
from google.colab import userdata


os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader

loader_csv = CSVLoader(file_path="/content/drive/MyDrive/Colab Notebooks/questions.csv")
data=loader_csv.load()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
#splitting the text into
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(data)

len(texts)

37

In [ ]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-base-en"
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity

bge_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs={'device': 'cpu'},
    encode_kwargs=encode_kwargs
)

In [ ]:
from langchain_chroma import Chroma

persist_directory = 'db'

## Here you can change the embeddings etc
embedding = bge_embeddings

vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

In [ ]:
retriever = vectordb.as_retriever(search_kwargs={"k": 5})

Making Agents

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

from langchain_core.output_parsers import StrOutputParser
from langchain_core.output_parsers import JsonOutputParser

In [ ]:
from langchain_groq import ChatGroq

GROQ_LLM = ChatGroq(
            model="llama3-70b-8192",
        )

In [ ]:
#RAG Chain
rag_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\n

     <|eot_id|><|start_header_id|>user<|end_header_id|>
    QUESTION: {question} \n
    CONTEXT: {context} \n
    Answer:
    <|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["question","context"],
)

rag_prompt_chain = rag_prompt | GROQ_LLM | StrOutputParser()

QUESTION = """why am i here"""
CONTEXT = retriever.invoke(QUESTION)

result = rag_prompt_chain.invoke({"question": QUESTION, "context":CONTEXT})

print(result)

I don't know why you are here as the provided context does not contain any information related to your question. The context appears to be a collection of questions and answers about various topics, including microphones and camera rentals, but it does not provide any information about your personal presence or purpose.


In [ ]:
rag_chain = (
    {"context": retriever , "question": RunnablePassthrough()}
    | rag_prompt
    | GROQ_LLM
    | StrOutputParser()
)

In [ ]:
rag_chain.invoke("what is the rquipment is damaged?")

'You may be liable for repair or replacement costs depending on the extent of the damage. Additionally, damage fees may be charged. Please contact the rental company immediately if the equipment is damaged.'

<h1>Basic chains<h1/>

* Categorize Emails +
* search Items +
* inquiry email return +
* Sentiment check +
* thanks email +
* CRM system +
* RAG Questions +


In [ ]:
#Categorize EMAIL
categoize_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are the Email Categorizer Agent for the film equipment rental service,You are a master at \
    understanding what a customer wants when they write an email and are able to categorize \
    it in a useful way. \

     <|eot_id|><|start_header_id|>user<|end_header_id|>
    Conduct a comprehensive analysis of the email provided and categorize into one of the following categories:
        availability_inquiry - used when someone is asking about the availability of equipments \
        customer_review - used when someone is conveying their experience either giving a positive review or complaining about something, they areproviding their feedback \
        product_enquiry - used when someone is asking for information about a product feature, benefit or service but not about pricing \\
        off_topic - when it doesnt relate to any other category \


            Output a single category only from the types ('availability_inquiry', 'customer_review', 'product_enquiry', 'off_topic') \
            eg:
            'availability_inquiry' \

    EMAIL CONTENT:\n\n {initial_email} \n\n
    <|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["initial_email"],
)

email_category_generator = categoize_prompt | GROQ_LLM | StrOutputParser()

EMAIL = """HI there, \n
do you have any new cameras \n

I am looking for new experiences.

Thanks,
Paul
"""

result = email_category_generator.invoke({"initial_email": EMAIL})

print(result)

'product_enquiry'


In [ ]:
## RAG QUESTIONS
search_rag_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are a master at working out the best questions to ask our knowledge agent to get the best info for the customer.

    given the INITIAL_EMAIL. Work out the best questions that will find the best \
    info for helping to write the final email. Remember when people ask about a renting they are \
    probably reffering to the film equipment renting. Write the questions to our knowledge system not to the customer.

    Return a JSON with a single key 'questions' with no more than 3 strings of and no premable or explaination.
    Make sure you are returning a JSON only

    <|eot_id|><|start_header_id|>user<|end_header_id|>
    INITIAL_EMAIL: {initial_email} \n
    <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["initial_email"],
)

search_rag = search_rag_prompt | GROQ_LLM | JsonOutputParser()



print(search_rag.invoke({"initial_email": EMAIL}))

{'questions': ['What are the steps to report damaged equipment during a rental?', 'What is the process for extending a rental period?', "What are the customer's responsibilities for equipment damage during a rental?"]}


In [ ]:
## Item searcher
search_item_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are the best at knowing what a person wants,

    given the INITIAL_EMAIL. Work out the film equipments the email sender wants. \
     Remember when people ask about a renting they are probably reffering to the film equipment renting.\
    Remember tthe items that you give should be inside the INITIAL_EMAIL

    Return a JSON with a single key 'items' with no premable or explaination.

    <|eot_id|><|start_header_id|>user<|end_header_id|>
    INITIAL_EMAIL: {initial_email} \n
    <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["initial_email"],
)

search_item_ = search_item_prompt | GROQ_LLM | JsonOutputParser()

EMAIL = """HI there, \n
do you have any new Canon EF 24-70mm f/2.8L II \n
, Atomos Ninja V and Aputure Light Dome II\n
I also want a burger please.

I am looking for new experiences.

Thanks,
Paul
"""

print(search_item_.invoke({"initial_email": EMAIL}))

{'items': ['Canon EF 24-70mm f/2.8L II', 'Atomos Ninja V', 'Aputure Light Dome II']}


In [ ]:
## Write Draft Email
write_inquiry_email_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are the Email Writer Agent for the Film Equipment Rental Service, take the INITIAL_EMAIL below \
    from a human that has emailed our company email address, items_asked that the human needs that item searcher gave \
    and the inventory_list_details with its details from the company inventory \
    write a helpful email in a thoughtful and friendly way.

            if the item the human asked is in inventory and is available then provide them with the item details including pricing.
            if the item the human asked is in inventory but is not available then apologize for the in availibilty of items and provide them with the similar items that are in our inventory remeber the items should be similar.
            if the item the human asked is not in inventory tell them that the item they need is not rented by the us and give them similar item recommendation from the inventory

            inventory_list_details is in the format of [(name, price),(name,price),...]
            You never make up information that hasn't been provided by the inventory_list_details or in the initial_email.
            make sure the recommendations you providehas to be inside the INVENTORY_LIST_DETAILS
            Always sign off the emails in appropriate manner and from Mohammed the Resident Manager.
            any items that is not related to the film equipment and is not present inside INVENTORY_LIST_DETAILS should be ignored.

            Return the email a JSON with a single key 'email_draft' and no premable or explaination.

    <|eot_id|><|start_header_id|>user<|end_header_id|>
    INITIAL_EMAIL: {initial_email} \n
    SEARCH_ITEM: {search_item} \n
    INVENTORY_LIST_DETAILS: {inventory_list_details} \n
    <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["initial_email","search_item","inventory_list_details"],
)

write_inquiry_email = write_inquiry_email_prompt | GROQ_LLM | JsonOutputParser()

search_item={'items': ['Canon EF 24-70mm f/2.8L II', 'Atomos Ninja V', 'Aputure Light Dome II']}
inventory_list_details=[('Canon EF 24-70mm f/2.8L II',1000)]


print(write_inquiry_email.invoke({"initial_email": EMAIL, "search_item":search_item,"inventory_list_details":inventory_list_details}))



{'email_draft': "Dear Paul,\n\nThank you for reaching out to us! We're excited to help you with your film equipment needs.\n\nWe do have the Canon EF 24-70mm f/2.8L II in our inventory, and it's available for rent at $1000.\n\nUnfortunately, we don't have the Atomos Ninja V and Aputure Light Dome II in our inventory. However, we can recommend similar items that might fit your needs. Please let us know if you're interested, and we'll be happy to provide you with some options.\n\nRegarding your request for a burger, I'm afraid we're a film equipment rental service and don't provide food services.\n\nIf you have any further questions or would like to proceed with the rental, please don't hesitate to reach out.\n\nBest regards,\nMohammed the Resident Manager"}


In [ ]:
## Item searcher
sentiment_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are the best at knowing what a person feels, you are an expert in sentiment analysis and you know what type of review people give\
    either positive or negative

    given the INITIAL_EMAIL. Find out the sentiment of the review. \
    remember the review should be either positive or negative.\
    if the person is happy with our serivices and items provided its a positive review
    if the person is unhappy with our services or item provided its a negative review

    Return a single word either positive or negative with no premable or explaination.
    eg: 'positive' .

    <|eot_id|><|start_header_id|>user<|end_header_id|>
    INITIAL_EMAIL: {initial_email} \n
    <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["initial_email"],
)

sentiment_ = sentiment_prompt | GROQ_LLM | StrOutputParser()

EMAIL = """HI there, \n
i was very happy with your services , but the items provided where bad.
but i was happy with the number of items provided.

I am looking for new experiences.

Thanks,
Paul
"""

print(sentiment_.invoke({"initial_email": EMAIL}))

negative


In [ ]:
## Write Draft Email
write_review_email_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are the Email Writer Agent for the Film Equipment Rental Service, take the INITIAL_EMAIL below \
    from a human that has emailed our company email address and the REVIEW from the sentiment analyzer\
    write a helpful email in a thoughtful and friendly way.

            if the review is positive then thank the sender and encorage them to share their experience on social media.
            if the review is negative then escaltae to CRM system for follow-up with a phone call from the customer service and offer a gift voucher in the reply.

            You never make up information that hasn't been provided in the initial_email.
            make sure the recommendations you provide has to be inside the limits.
            Always sign off the emails in appropriate manner and from Mohammed the Resident Manager.
            Apologize for the fault caused and provided relating gift voucher

            Return the email a JSON with a single key 'email_draft' and no premable or explaination.

    <|eot_id|><|start_header_id|>user<|end_header_id|>
    INITIAL_EMAIL: {initial_email} \n
    REVIEW: {review} \n
    <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["initial_email","review"],
)

write_review_email = write_review_email_prompt | GROQ_LLM | JsonOutputParser()
review='negative'


print(write_review_email.invoke({"initial_email": EMAIL, "review":review}))



{'email_draft': "Dear Paul,\n\nThank you for taking the time to share your feedback about our film equipment rental services. I'm sorry to hear that the items provided did not meet your expectations. I apologize for the inconvenience this has caused.\n\nI'd like to make things right and would appreciate the opportunity to discuss this further with you. I've escalated your concern to our customer service team, and they will be in touch with you via phone call to understand the issue better and provide a suitable solution.\n\nAs a gesture of goodwill, we'd like to offer you a gift voucher for your next rental. We value your business and would like to ensure that you have a better experience with us in the future.\n\nThank you for your patience and understanding.\n\nBest regards,\nMohammed, Resident Manager"}


In [ ]:
## Write Draft Email
write_CS_email_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are the Email Writer Agent for the Film Equipment Rental Service, take the INITIAL_EMAIL below \
    from a human that has emailed our company email address and the REVIEW from the sentiment analyzer\
    write a helpful email and professional email to the cusotmer service side.
    Remember you are emailing your own coworker in the customer service side.

            if negative review is provided then forward the email by the client and ask the customer service to handle it.
            if no review is provided then forward it as the email might be more concerned towards customer service.
            if review is postive return null string
            You never make up information that hasn't been provided in the initial_email.
            make sure the recommendations you provide has to be inside the limits.
            Always sign off the emails in appropriate manner and from Mohammed the Resident Manager.
            Ask the customer service to handle it professionally.
            remember if the review is positive the string returned should be empty.
            Return the email a JSON with a single key 'email_draft' and no premable or explaination.

    <|eot_id|><|start_header_id|>user<|end_header_id|>
    INITIAL_EMAIL: {initial_email} \n
    REVIEW: {review} \n
    <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["initial_email","review"],
)

write_CS_email = write_CS_email_prompt | GROQ_LLM | JsonOutputParser()
review='positive'


print(write_CS_email.invoke({"initial_email": EMAIL, "review":review}))



{'email_draft': ''}


In [ ]:
## Write Draft Email
write_RAG_email_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are the Email Writer Agent for the Film Equipment Rental Service, take the INITIAL_EMAIL below \
    from a human that has emailed our company email address and RAG_QUESTIONS that are given by the analylist\
    write a helpful email and professional email to the cusotmer service side.
    Remember only answer the questions that are asked.

            if proper answer is given to questions, then email the answers accuratley
            if proper answers are not provided escalate the issue to customer service

            You never make up information that hasn't been provided in the initial_email.
            Always sign off the emails in appropriate manner and from Mohammed the Resident Manager.

            Return the email a JSON with a single key 'email_draft' and no premable or explaination.

    <|eot_id|><|start_header_id|>user<|end_header_id|>
    INITIAL_EMAIL: {initial_email} \n
    RAG_QUESTIONS: {questions} \n
    <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["initial_email","questions"],
)

write_RAG_email = write_RAG_email_prompt | GROQ_LLM | JsonOutputParser()
review=['What are the latest camera models available for rent?', 'What kind of camera equipment is recommended for beginners looking to try new experiences?', 'What are some popular camera rentals for creative projects or unique experiences?']


print(write_RAG_email.invoke({"initial_email": EMAIL, "questions":review}))



{'email_draft': "Dear Paul,\n\nThank you for your feedback about our services. We apologize for the issues you experienced with the items provided and appreciate your positive comment about the quantity of items.\n\nRegarding your interest in new experiences, we'd be happy to help you explore our range of camera equipment. \n\nTo answer your questions, our latest camera models available for rent include the Sony A7S III, Canon EOS R5, and Nikon Z9. For beginners looking to try new experiences, we recommend starting with a basic DSLR kit or a mirrorless camera with a versatile lens.\n\nAs for popular camera rentals for creative projects or unique experiences, our clients have been enjoying the DJI Ronin 4D, ARRI ALEXA Mini, and the RED Komodo. These cameras offer advanced features and exceptional image quality.\n\nIf you'd like to discuss your project or experience further, please don't hesitate to reach out. We're here to help.\n\nBest regards,\nMohammed, Resident Manager"}


In [ ]:
from langchain.schema import Document
from langgraph.graph import END, StateGraph

In [ ]:
from typing_extensions import TypedDict
from typing import List,Tuple

### State

class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        initial_email: email
        email_category: email category

        final_email: LLM generation

        num_steps: number of steps
    """
    initial_email : str
    email_category : str
    final_email : str
    CS_email : str
    rag_results : List[str] # this will now be the RAG results
    sentiment : str
    num_steps : int
    search_item : List[str]
    rag_questions : List[str]
    inventory_list_details : List[Tuple[str,int]]

<h1>Nodes<h1/>


* categorize_email
* check_items
* Inquery_handling
* analyze_draft_email
* rewrite_email
* Sentiment_email
* CS_email
* question_handling
   



In [ ]:
def categorize_email(state):
    """take the initial email and categorize it"""
    print("---CATEGORIZING INITIAL EMAIL---")
    initial_email = state['initial_email']
    num_steps = int(state['num_steps'])
    num_steps += 1

    email_category = email_category_generator.invoke({"initial_email": initial_email})
    print(email_category)

    return {"email_category": email_category, "num_steps":num_steps}

In [ ]:

def question_handling(state):

    print("--- INFO RAG---")
    initial_email = state["initial_email"]
    email_category = state["email_category"]
    num_steps = state['num_steps']
    num_steps += 1

    # Web search
    questions = search_rag.invoke({"initial_email": initial_email})
    questions = questions['questions']
    # print(questions)
    rag_results = []
    for question in questions:
        print(question)
        temp_docs = rag_chain.invoke(question)
        print(temp_docs)
        question_results = question + '\n\n' + temp_docs + "\n\n\n"
        if rag_results is not None:
            rag_results.append(question_results)
        else:
            rag_results = [question_results]

    return {"rag_results": rag_results,"rag_questions":questions, "num_steps":num_steps}

In [ ]:
def sentiment_email(state):
    """takes email as input, finds sentiments and gives sentiment email as an output"""
    print("---FINDING SENTIMENTS---")
    initial_email = state['initial_email']
    num_steps = int(state['num_steps'])
    num_steps += 1

    sentiment=sentiment_.invoke({"initial_email": EMAIL})


    return {"num_steps":num_steps,"sentiment":sentiment}

In [ ]:
def review_email(state):
    """takes enail as input, finds sentiments and gives sentiment email as an output"""
    print("---getting the review email---")
    initial_email = state['initial_email']
    review=state['sentiment']
    num_steps = int(state['num_steps'])
    num_steps += 1

    email=write_review_email.invoke({"initial_email": EMAIL, "review":review})


    return {"num_steps":num_steps,"final_email":email["email_draft"]}

In [ ]:
def Cs_email(state):
    """mail to the customer service department"""
    print("---getting the customer service email---")
    initial_email = state['initial_email']
    review=state['sentiment']
    num_steps = int(state['num_steps'])
    num_steps += 1


    CS_email=write_CS_email.invoke({"initial_email": EMAIL, "review":review})

    return {"num_steps":num_steps,"CS_email":CS_email["email_draft"]}

In [ ]:
def get_items(state):
    """mail to the customer service department"""
    print("---getting the customer service email---")
    initial_email = state['initial_email']
    num_steps = int(state['num_steps'])
    num_steps += 1
    import sqlite3
    conn = sqlite3.connect('/content/drive/MyDrive/Colab Notebooks/emaildb')
    search_item = search_item_.invoke({"initial_email": EMAIL})
    cursor = conn.cursor()
    cursor.execute('''SELECT name, price FROM rental_equipment''')
    inventory_list = cursor.fetchall()

    return {"num_steps":num_steps,"search_item":search_item,"inventory_list_details":inventory_list}

In [ ]:
def inquery_email(state):
    """mail to get the price list or similar recommendation"""
    print("---getting the inquery email---")
    initial_email = state['initial_email']
    search_item = state['search_item']
    inventory_list_details=state['inventory_list_details']
    num_steps = int(state['num_steps'])
    num_steps += 1


    final_email=write_inquiry_email.invoke({"initial_email": EMAIL, "search_item":search_item,"inventory_list_details":inventory_list_details})

    return {"num_steps":num_steps,"final_email":final_email["email_draft"]}

In [ ]:

def write_rag_email(state):
    """this finally forms the proper answer emal for rag"""
    print("---getting rag email---")
    initial_email = state['initial_email']
    rag_questions=state['rag_questions']
    num_steps = int(state['num_steps'])
    num_steps += 1

    final_email=write_RAG_email.invoke({"initial_email": EMAIL, "questions":rag_questions})
    return {"num_steps":num_steps,"final_email":final_email["email_draft"]}

<h1>Conditional edges<h1/>

In [ ]:
def category_seperator(state):
    """
    Route email to web search or not.
    Args:
        state (dict): The current graph state
    Returns:
        str: Next node to call
    """

    print("---ROUTE TO catgoried email---")
    email_category = state["email_category"]
    print(f"Email category: {email_category} (Type: {type(email_category)})")

    if 'availability_inquiry' in email_category:
        print("---ROUTE EMAIL TO get_items---")
        return "get_items_"
    elif 'customer_review' in email_category :
        print("---ROUTE EMAIL TO sentment email---")
        return "sentment_email_"
    elif 'product_enquiry' in email_category :
        print("---ROUTE EMAIL TO question handling---")
        return "question_handling_"
    elif 'off_topic' in email_category :
      print("---ROUTE EMAIL TO cs_email---")
      return "cs_email_"

<h1>Build Graph<h1/>
<h3>Add node<h3/>

In [ ]:
workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("categorize_email", categorize_email)
workflow.add_node("get_items", get_items)
workflow.add_node("sentiment_email", sentiment_email)
workflow.add_node("question_handling", question_handling)
workflow.add_node("Cs_email", Cs_email)
workflow.add_node("inquery_email", inquery_email)
workflow.add_node("review_email", review_email)
workflow.add_node("write_rag_email", write_rag_email)

<h3>Add edge<h3/>

In [ ]:
workflow.set_entry_point("categorize_email")

workflow.add_conditional_edges(
    "categorize_email",
    category_seperator,
    {
        "get_items_": "get_items",
        "sentment_email_": "sentiment_email",
        "question_handling_": "question_handling",
        "cs_email_": "Cs_email",
    },
)
workflow.add_edge("get_items", "inquery_email")
workflow.add_edge("sentiment_email", "review_email")
workflow.add_edge("review_email", "Cs_email")
workflow.add_edge("question_handling", "write_rag_email")
workflow.add_edge("inquery_email",END)
workflow.add_edge("Cs_email",END)
workflow.add_edge("write_rag_email",END)

In [ ]:

# Compile
app = workflow.compile()

In [ ]:
EMAIL="""HI there, \n
I am writing this email to express my dissatifaction for the products i rented.
The products where unclean and unhygenic.The products were also overused.
please see to it.

Thanks,
Ringo
"""

In [ ]:
inputs = {"initial_email": EMAIL, "num_steps":0}
for output in app.stream(inputs):
    for key, value in output.items():
        pprint(f"Finished running: {key}:")

---CATEGORIZING INITIAL EMAIL---
'customer_review'
---ROUTE TO catgoried email---
Email category: 'customer_review' (Type: <class 'str'>)
---ROUTE EMAIL TO sentment email---
'Finished running: categorize_email:'
---FINDING SENTIMENTS---
'Finished running: sentiment_email:'
---getting the review email---
'Finished running: review_email:'
---getting the customer service email---
'Finished running: Cs_email:'


In [ ]:
output = app.invoke(inputs)

print(output)

---CATEGORIZING INITIAL EMAIL---
'customer_review'
---ROUTE TO catgoried email---
Email category: 'customer_review' (Type: <class 'str'>)
---ROUTE EMAIL TO sentment email---
---FINDING SENTIMENTS---
---getting the review email---
---getting the customer service email---
{'initial_email': 'HI there, \n\nI am writing this email to express my dissatifaction for the products i rented.\nThe products where unclean and unhygenic.The products were also overused.\nplease see to it.\n\nThanks,\nRingo\n', 'email_category': "'customer_review'", 'final_email': 'Dear Ringo,\n\nI am so sorry to hear that the products you rented from us did not meet your expectations. I apologize for the inconvenience and distress caused by the unclean and unhygienic condition of the equipment. I understand how frustrating it must be to receive overused products.\n\nI want to assure you that we take situations like this very seriously and I have escalated your concern to our CRM system for immediate follow-up. Our cus

In [ ]:
output.keys()

dict_keys(['initial_email', 'email_category', 'final_email', 'CS_email', 'sentiment', 'num_steps'])

In [ ]:
if output['final_email']:
  print(output['final_email'])
if output['CS_email']:
  print(output['CS_email'])

Dear Ringo,

I am so sorry to hear that the products you rented from us did not meet your expectations. I apologize for the inconvenience and distress caused by the unclean and unhygienic condition of the equipment. I understand how frustrating it must be to receive overused products.

I want to assure you that we take situations like this very seriously and I have escalated your concern to our CRM system for immediate follow-up. Our customer service team will be in touch with you shortly to discuss this further and provide a resolution.

As a token of our apology, we would like to offer you a gift voucher for your next rental. We value your business and would like to assure you that we are taking necessary steps to prevent such incidents in the future.

Thank you for bringing this to our attention.

Best regards,
Mohammed
Resident Manager
Subject: Concern with Recent Rental Experience

Dear Customer Service Team,

I am forwarding an email from Ringo, who has expressed dissatisfaction 